In [ ]:
!pip install python-docx
!pip install PyPDF2

In [ ]:
# upload the CSV file into the notebook
from google.colab import files
uploaded = files.upload()

In [ ]:
pip install pymilvus

In [ ]:
import gradio as gr
import PyPDF2, docx, json, random
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from pymilvus import connections, Collection


# -----------------------------
# 1️⃣ Connect to Zilliz Cloud
# -----------------------------
connections.connect(
    alias="default",
    uri="https://in03-feb569ec82b1b76.serverless.aws-eu-central-1.cloud.zilliz.com",  # your URI
    token="520e0e883ef97fcc4663dca8514090a2a491dd29b714711e961807fdbff8a163d82abd308cf1a8ef546698eb1759aa7054cbc295")


collection = Collection("interveiw_Knowledge")
embedder = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
# -----------------------------
# 2️⃣ Load LLM (small one for demo)
# -----------------------------


# Load LLM (small one for demo)
chatbot_model = pipeline(
    "text-generation",
    model="HuggingFaceH4/zephyr-7b-beta",
    device_map="auto",
    torch_dtype="auto"
)


In [ ]:
import gradio as gr
import PyPDF2, docx, json, random
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from pymilvus import connections, Collection
import random

# Store sessions for mock interview
interview_sessions = {}

# -----------------------------
# 3️⃣ File Extractors
# -----------------------------

def extract_text_from_pdf(file_obj):
    text = ""
    reader = PyPDF2.PdfReader(file_obj)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text.strip()


def extract_text_from_docx(file_obj):
    doc = docx.Document(file_obj)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])


def extract_text(file_obj):
    name = file_obj.name
    if name.endswith(".pdf"):
        return extract_text_from_pdf(file_obj)
    elif name.endswith(".docx"):
        return extract_text_from_docx(file_obj)
    return ""


# -----------------------------
# 4️⃣ RAG Retrieval Function
# -----------------------------
def retrieve_questions_from_zilliz(cv_text, jd_text, top_k=5):
    # If user uploads blank or short files
    if not cv_text.strip() or not jd_text.strip():
        return get_random_questions_from_collection(top_k)

    # Create a meaningful semantic search query
    query = f"{jd_text[:500]} related interview questions for candidate with {cv_text[:500]}"
    query_vector = embedder.encode([query]).tolist()

    search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}

    results = collection.search(
        data=query_vector,
        anns_field="embedding",
        param=search_params,
        limit=50,
        output_fields=["text"]
    )

    # Extract retrieved question texts
    hits = [hit.entity.get("text") for hit in results[0] if hit.distance > 0]

    # Remove duplicates while preserving order
    hits = list(dict.fromkeys(hits))

    # Fallback if no results found
    if not hits:
        hits = get_random_questions_from_collection(top_k)

    # Randomize subset
    hits = random.sample(hits, min(top_k, len(hits)))
    return hits


def get_random_questions_from_collection(n=5):
    """Fetch random questions from the whole collection."""
    data = collection.query(expr="", output_fields=["text"], limit=100)
    all_qs = [d["text"] for d in data]
    return random.sample(all_qs, min(n, len(all_qs)))

# -----------------------------
# 5️⃣ Generate Summary + Questions
# -----------------------------
def generate_summary_and_questions(cv_file, jd_file):
    if not cv_file or not jd_file:
        return "⚠️ Please upload both a CV and Job Description.", []

    cv_text = extract_text(cv_file)
    jd_text = extract_text(jd_file)

    questions = retrieve_questions_from_zilliz(cv_text, jd_text, top_k=5)

    if not questions:
        return "No relevant questions found. Try a different job description.", []

    summary = (
        f"🎯 **Top {len(questions)} relevant questions retrieved from knowledge base:**\n\n"
        + "\n".join([f"{i+1}. {q}" for i, q in enumerate(questions)])
    )
    return summary, questions


# -----------------------------
# 6️⃣ Feedback Generator
# -----------------------------
def give_feedback(answer, question):
    prompt = (
        f"Evaluate the following candidate answer to a technical question.\n\n"
        f"Question: {question}\n"
        f"Candidate's Answer: {answer}\n\n"
        f"Your response must include only:\n"
        f"1️⃣ Evaluation: State 'Correct', 'Partially Correct', or 'Incorrect'.\n"
        f"2️⃣ Correct Answer: Write the ideal answer in 1–2 short sentences.\n"
        f"3️⃣ Review:\n"
        f"   - ✅ Strengths (1 short sentence)\n"
        f"   - ⚠️ Areas for improvement (1 short sentence)\n\n"
        f"Keep your total response under 100 words. Do NOT show any examples or repeat the question."
    )

    response = chatbot_model(
        prompt,
        max_new_tokens=150,
        do_sample=False,
        temperature=0.3,
        top_p=0.9
    )[0]["generated_text"]

    return response.replace(prompt, "").strip()



## -----------------------------
# 7️⃣ Mock Interview Logic (Final Fixed Version)
# -----------------------------
def start_interview(questions):
    """Start the interview and initialize session."""
    if not questions:
        return "⚠️ No questions available. Please upload CV + JD first.", {}

    session_data = {"questions": questions, "current": 0}
    first_q = questions[0]
    status_msg = f"🎤 **Starting personalized mock interview!**\n\n🧠 **First question:**\n{first_q}"
    return status_msg, session_data


def chat_with_bot(message, history, session_data):
    """Handle user responses and provide feedback."""
    if not session_data or "questions" not in session_data or not session_data["questions"]:
        return "⚠️ Please start the mock interview first.", session_data

    q_index = session_data.get("current", 0)
    questions = session_data["questions"]

    current_question = questions[q_index]
    feedback = give_feedback(message, current_question)

    q_index += 1
    if q_index < len(questions):
        session_data["current"] = q_index
        next_q = questions[q_index]
        progress = f"({q_index}/{len(questions)})"
        reply = f"💬 **Feedback:** {feedback}\n\n➡️ **Next question {progress}:** {next_q}"
    else:
        reply = f"💬 **Final Feedback:** {feedback}\n\n✅ Interview finished. Great job!"
        session_data = {}  # reset after interview

    return reply, session_data


# -----------------------------
# 8️⃣ Gradio Interface
# -----------------------------
with gr.Blocks(theme=gr.themes.Soft(), title="RAG Mock Interview Coach") as demo:
    gr.Markdown("## 🤖 CV + JD Powered Mock Interview Coach (Now with RAG & Zilliz Cloud)")

    with gr.Tabs():
        # Tab 1: Upload & Analyze
        with gr.Tab("Upload & Analyze"):
            gr.Markdown("### 📄 Upload your CV and Job Description to generate relevant questions.")
            with gr.Row():
                cv_upload = gr.File(label="Upload CV (PDF or DOCX)", type="filepath")
                jd_upload = gr.File(label="Upload Job Description (PDF or DOCX)", type="filepath")

            output = gr.Markdown(label="Extracted Info & Questions")
            role_questions = gr.State([])

            generate_btn = gr.Button("🔍 Analyze CV + JD & Retrieve Questions")
            generate_btn.click(
                fn=generate_summary_and_questions,
                inputs=[cv_upload, jd_upload],
                outputs=[output, role_questions]
            )

        # Tab 2: Mock Interview
        with gr.Tab("Mock Interview"):
            gr.Markdown("### 🎤 Start Your Personalized Mock Interview")

            # Session state to persist across chat
            session_state = gr.State({})

            start_btn = gr.Button("🚀 Start Mock Interview")
            start_output = gr.Markdown(label="Interview Status")

            # Start interview properly — split text + state
            def start_and_show(questions):
                message, session = start_interview(questions)
                return message, session

            start_btn.click(
                fn=start_and_show,
                inputs=[role_questions],
                outputs=[start_output, session_state]
            )

            chatbot = gr.ChatInterface(
                fn=chat_with_bot,
                title="Interview Coach",
                description="💬 Answer the question, and get instant feedback!",
                additional_inputs=[session_state],
                additional_outputs=[session_state]
            )

demo.launch()

In [ ]:
import streamlit as st
import PyPDF2, docx
from sentence_transformers import SentenceTransformer
from pymilvus import connections, Collection
import random

# Connect to Milvus and load collection as before
connections.connect(
    alias="default",
    uri="https://in03-feb569ec82b1b76.serverless.aws-eu-central-1.cloud.zilliz.com",
    token="520e0e883ef97fcc4663dca8514090a2a491dd29b714711e961807fdbff8a163d82abd308cf1a8ef546698eb1759aa7054cbc295"
)
collection = Collection("interveiw_Knowledge")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def extract_text_from_pdf(file_obj):
    text = ""
    reader = PyPDF2.PdfReader(file_obj)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text.strip()

def extract_text_from_docx(file_obj):
    doc = docx.Document(file_obj)
    return "\n".join([p.text for p in doc.paragraphs if p.text.strip()])

def extract_text(file_obj):
    name = file_obj.name
    if name.endswith(".pdf"):
        return extract_text_from_pdf(file_obj)
    elif name.endswith(".docx"):
        return extract_text_from_docx(file_obj)
    return ""

def retrieve_questions_from_zilliz(cv_text, jd_text, top_k=5):
    if not cv_text.strip() or not jd_text.strip():
        return get_random_questions_from_collection(top_k)
    query = f"{jd_text[:500]} related interview questions for candidate with {cv_text[:500]}"
    query_vector = embedder.encode([query]).tolist()
    search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}
    results = collection.search(
        data=query_vector,
        anns_field="embedding",
        param=search_params,
        limit=50,
        output_fields=["text"]
    )
    hits = [hit.entity.get("text") for hit in results[0] if hit.distance > 0]
    hits = list(dict.fromkeys(hits))
    if not hits:
        hits = get_random_questions_from_collection(top_k)
    hits = random.sample(hits, min(top_k, len(hits)))
    return hits

def get_random_questions_from_collection(n=5):
    data = collection.query(expr="", output_fields=["text"], limit=100)
    all_qs = [d["text"] for d in data]
    return random.sample(all_qs, min(n, len(all_qs)))

# Streamlit UI
st.title("RAG Mock Interview Coach (Streamlit Edition)")
st.write("Upload your CV and Job Description to generate relevant interview questions.")

cv_file = st.file_uploader("Upload CV (PDF or DOCX)", type=["pdf", "docx"])
jd_file = st.file_uploader("Upload Job Description (PDF or DOCX)", type=["pdf", "docx"])

if st.button("Analyze CV + JD & Retrieve Questions"):
    if cv_file and jd_file:
        cv_text = extract_text(cv_file)
        jd_text = extract_text(jd_file)
        questions = retrieve_questions_from_zilliz(cv_text, jd_text, top_k=5)
        if questions:
            st.markdown("### Top Relevant Questions")
            for i, q in enumerate(questions):
                st.write(f"{i+1}. {q}")
        else:
            st.warning("No relevant questions found. Try a different job description.")
    else:
        st.warning("Please upload both a CV and Job Description.")


In [ ]:
!pip install streamlit

Building RAG

In [ ]:
!pip install pymilvus


In [ ]:
url = "in03-feb569ec82b1b76.serverless.aws-eu-central-1.cloud.zilliz.com"
token = "520e0e883ef97fcc4663dca8514090a2a491dd29b714711e961807fdbff8a163d82abd308cf1a8ef546698eb1759aa7054cbc295"

In [ ]:
from pymilvus import connections

connections.connect(
    alias="default",
    uri="https://in03-feb569ec82b1b76.serverless.aws-eu-central-1.cloud.zilliz.com",
    token="520e0e883ef97fcc4663dca8514090a2a491dd29b714711e961807fdbff8a163d82abd308cf1a8ef546698eb1759aa7054cbc295"
)

In [ ]:
import ast
from pymilvus import Collection, connections
from sentence_transformers import SentenceTransformer

# -----------------------------
# 2️⃣ Load Collection
# -----------------------------
collection = Collection("interveiw_Knowledge")
collection.load()  # Make sure the collection is loaded into memory[web:94]

# -----------------------------
# 3️⃣ Query All Data (Direct Access)
# -----------------------------
# Fetch up to 100 records from the collection
data = collection.query(expr="", output_fields=["role", "skill", "question", "embedding"], limit=100)
print("Sample data from collection:")
for item in data:
    print(item)

# -----------------------------
# 4️⃣ Semantic Search (Vector Search)
# -----------------------------
embedder = SentenceTransformer("all-MiniLM-L6-v2")
query_text = "Sample semantic search query"
query_vector = embedder.encode([query_text]).tolist()

search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}

results = collection.search(
    data=query_vector,
    anns_field="embedding",  # The name of your vector field
    param=search_params,
    limit=5,
    output_fields=["role", "skill", "question"]
)

print("Semantic search results:")
for hit in results[0]:
    print(hit.entity)

# -----------------------------
# 5️⃣ Get All Questions (for Sampling)
# -----------------------------
questions_data = collection.query(expr="", output_fields=["question"], limit=100)

# Convert each string to a dictionary
#all_questions = [ast.literal_eval(d)["text"] for d in questions_data]
#print(all_questions)
all_questions = [d["text"] for d in questions_data if "text" in d]
print("All questions:", all_questions)

In [ ]:
print(collection.schema)


In [ ]:
questions_data = collection.query(expr="", output_fields=["text"], limit=100)
print(questions_data)


1. Quantitative Results (Evaluation Metrics)

a) Accuracy / Correctness

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

ref_answer = "An INNER JOIN returns only rows with matching values in both tables."
gen_answer = "INNER JOIN gives rows where both tables have matching data."

emb_ref = model.encode(ref_answer, convert_to_tensor=True)
emb_gen = model.encode(gen_answer, convert_to_tensor=True)

similarity = util.cos_sim(emb_ref, emb_gen)
print(f"Semantic Similarity: {similarity.item():.3f}")


✅ Interpretation:
If similarity > 0.8 → Good answer
0.6–0.8 → Partial
<0.6 → Poor or irrelevant

b) Precision / Recall / F1 (if you classify responses)

In [ ]:
from sklearn.metrics import classification_report

y_true = ["correct", "correct", "incorrect", "partial"]
y_pred = ["correct", "partial", "incorrect", "correct"]

print(classification_report(y_true, y_pred))


c) BLEU or ROUGE (text similarity)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

reference = ["An INNER JOIN returns rows with matching values in both tables"]
candidate = "INNER JOIN gives rows where both tables have matching data"

# Tokenize correctly
reference = [ref.split() for ref in reference]   # List of tokenized reference sentences
candidate = candidate.split()                    # Tokenized candidate

bleu_score = sentence_bleu(reference, candidate)
print(f"BLEU Score: {bleu_score:.3f}")


✅ BLEU closer to 1 means closer to the reference answer.

d) Latency (Response Time)

In [ ]:
import time

start = time.time()
# Call your RAG answer function here
end = time.time()

print(f"Response time: {end - start:.2f} seconds")


In [ ]:
import time
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer, util

# Load embedding model for semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
smooth = SmoothingFunction().method1

def evaluate_response(reference_answer, model_answer):
    # BLEU (with smoothing)
    reference = [reference_answer.split()]
    candidate = model_answer.split()
    bleu = sentence_bleu(reference, candidate, smoothing_function=smooth)

    # Semantic similarity
    emb1 = model.encode(reference_answer, convert_to_tensor=True)
    emb2 = model.encode(model_answer, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(emb1, emb2).item()

    return bleu, similarity


def evaluate_system(data):
    """
    data: list of dicts with keys:
      'question', 'reference_answer', 'model_answer', 'response_time'
    """
    results = []

    for item in data:
        bleu, sim = evaluate_response(item['reference_answer'], item['model_answer'])
        results.append({
            'Question': item['question'],
            'BLEU': bleu,
            'Semantic_Similarity': sim,
            'Response_Time': item['response_time']
        })

    df = pd.DataFrame(results)
    print("\n--- Summary ---")
    print(df.describe())
    df.to_csv("evaluation_results.csv", index=False)
    print("\n✅ Results saved to evaluation_results.csv")
    return df


In [ ]:
from google.colab import files
files.download("evaluation_results.csv")


In [ ]:
import os
print(os.listdir())


In [ ]:
# from evaluate_results import evaluate_response # This import is not needed as the function is defined in this notebook
import time

# Example flow inside your Gradio callback:
start = time.time()
# generated_answer = model_pipeline(question)   # your Zephyr-7B response # This variable is not defined
end = time.time()

response_time = end - start

# Compare with your reference answer (ground truth)
# bleu, semantic_sim = evaluate_response(reference_answer, generated_answer) # reference_answer and generated_answer are not defined

# print(f"BLEU: {bleu:.3f}, Semantic Similarity: {semantic_sim:.3f}, Time: {response_time:.2f}s")
print(f"Response time: {response_time:.2f} seconds")